<a href="https://colab.research.google.com/github/vineetbiyani09/NLP/blob/master/NLP_7_NER_Assignment_J010.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

import nltk
import re
import random
import string
from string import punctuation
from nltk.corpus import stopwords # Words which do not contribute to the sentiment analysis
from nltk.tokenize import word_tokenize # Separating sentences in different components, every word and punctuation
from nltk.stem import LancasterStemmer # Converts the words to root words which might or might not make sense
from nltk.corpus import twitter_samples # Load the twitter dataset
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk import FreqDist
from nltk import classify
from nltk import NaiveBayesClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
import xgboost as xgb
# from catboost import CatBoostClassifier
# from lightgbm import LGBMClassifier
from tqdm import tqdm, trange
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
! pip install -q kaggle

In [3]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vineetbiyani","key":"3ca3d9fb43ac9952812c7dc33cdfd696"}'}

In [4]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              221KB  2021-03-30 07:16:41           1346  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-02-17 16:10:44            964  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   1GB  2021-02-18 10:08:27            794  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19            353  
alsgroup/end-als                                            End ALS Kaggle C

In [7]:
! kaggle datasets download -d abhinavwalia95/entity-annotated-corpus

 34% 9.00M/26.4M [00:00<00:00, 37.8MB/s]
100% 26.4M/26.4M [00:00<00:00, 76.0MB/s]


In [8]:
! unzip entity-annotated-corpus.zip

Archive:  entity-annotated-corpus.zip
  inflating: ner.csv                 
replace ner_dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ner_dataset.csv         


In [9]:
data = pd.read_csv("/content/ner_dataset.csv",encoding = 'latin-1')

In [10]:
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [11]:
class sentenceID(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        function = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(function)
        self.sentences = [s for s in self.grouped]
    
    def retrieve(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [12]:
data = data.fillna(method="ffill")
Sentences = sentenceID(data)
Sentences_list = [" ".join([s[0] for s in sent]) for sent in Sentences.sentences]
Sentences_list[:1] #First sentence
Sentences_list[99:100] #100th sentence

["The mysterious female leader was a Tzotzil Indian and a promoter of women 's rights ."]

In [13]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [14]:
Sentences_list_a = Sentences_list[:7500]
listToStr_a = ' '.join([str(elem) for elem in Sentences_list_a])
article = nlp(listToStr_a)
labels = [x.label_ for x in article.ents]
items = [x.text for x in article.ents]
print(items)
Counter(items).most_common(3)

['Thousands', 'London', 'Iraq', 'British', 'Iranian', 'Wednesday', 'IAEA', 'Saturday', 'Orakzai', 'Taliban', 'South Waziristan', 'a tense hour-long', 'U.N.', 'Jan Egeland', 'Sunday', 'U.S.', 'Indonesian', 'Australian', 'western Aceh province', 'Egeland', '1.8 million', 'Indonesia', 'Sri Lanka', 'Maldives', 'India', "last week 's", 'millions', 'Asia', 'Africa', 'Some 1,27,000', 'U.N.', 'Lebanese', 'Friday', 'Christian', 'Beirut', 'Beirut', 'United Nations', 'New York', 'Fouad Siniora', 'Lebanese', 'One', 'more than 20', 'late Friday', 'Lebanon', 'February', 'Rafik Hariri', '20', 'Syria', 'Friday', 'days', 'U.N.', 'Detlev Mehlis', 'Damascus', 'Syrian', 'Iceland', 'Israeli', 'Ariel Sharon', 'Thursday', 'last month', 'Sharon', 'Sharon', 'Sharon', 'December 25 , one week', 'first', '$ 10 million', 'SpaceShipOne', 'Burt Rutan', 'Ansari', 'Saturday', 'U.S.', 'Missouri', 'SpaceShipOne', 'two-week', 'at least 100 kilometers', 'Earth', 'late September', 'early October', 'California', 'Mojave', '

[('U.S.', 549), ('two', 303), ('Iraq', 275)]

In [15]:
sentences = [x for x in article.sents]
print(sentences[0])
print(sentences[1000])

Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .
In an interview broadcast Monday , Syrian President Bashar al-Assad said he is ready to hold talks with Israel but that an impartial arbiter should mediate between the two sides .


In [16]:
displacy.render(nlp(str(sentences[20])), jupyter = True, style = 'ent')

/usr/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


In [17]:
displacy.render(nlp(str(sentences[0])), jupyter = True, style = 'ent')
displacy.render(nlp(str(sentences[1000])), jupyter = True, style = 'ent')

In [18]:
displacy.render(nlp(str(sentences[0])), style='dep', jupyter = True, options = {'distance': 120})

In [19]:
displacy.render(nlp(str(sentences[1000])), style='dep', jupyter = True, options = {'distance': 120})
